# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import os
path_input = os.path.join(os.getcwd(), "sample_data", "machine_type_1", "id_00")
path_input

'/workspaces/atelier/examples/sample_data/machine_type_1/id_00'

## Reading Data Files

In [4]:
from mtsa import calculate_aucroc

2023-01-11 19:19:05.735067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from mtsa import files_train_test_split
X_train, X_test, y_train, y_test = files_train_test_split(path_input)
y_train

array([1., 1., 1., 1., 1.])

## MFFCMix Model

In [6]:
from mtsa import MFCCMix
model_mfccmix = MFCCMix()
model_mfccmix.fit(X_train, y_train)
model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [7]:
auc = calculate_aucroc(model_mfccmix, X_test, y_test)
auc

0.8400000000000001

## Hitachi Model

In [8]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

2023-01-11 19:19:13.590546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: UNKNOWN ERROR (34)
2023-01-11 19:19:13.590619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9c382bd775b6): /proc/driver/nvidia/version does not exist
2023-01-11 19:19:13.590812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


4/4 [==============================] - 1s 62ms/step - loss: 3234.0093 - val_loss: 3174.2354


Pipeline(steps=[('wav2array', Wav2Array(mono=False)),
                ('demux2array', Demux2Array()),
                ('array2melspec',
                 Array2MelSpec(frames=5, hop_length=512, n_fft=1024, n_mels=64,
                               power=2.0, sampling_rate=None)),
                ('final_model',
                 <tsa.models.hitachi.AutoEncoderMixin object at 0x7f16b6d05dc0>)])

In [9]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

14/14 [==============================] - 0s 2ms/step


0.32000000000000006